In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from tqdm import tqdm


/Volumes/home/Learn/Master/3. Multimedia/AIChallenge/Code/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")  

/Volumes/home/Learn/Master/3. Multimedia/AIChallenge/Code/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:


img_url = 'keyframes/L01_V001/001.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_image = Image.open(img_url).convert('RGB')

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


/Volumes/home/Learn/Master/3. Multimedia/AIChallenge/Code/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a city skyline with a large logo in the middle
a close up of a city skyline with a large orange sign


In [4]:
text = "In the image includes"
inputs = processor(raw_image, text, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


in the image includes a city skyline and a river with boats


In [5]:

img_url = 'keyframes/L01_V001/016.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_image = Image.open(img_url).convert('RGB')

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt")
out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

a photography of a flooded street with a truck driving through it


In [6]:
from langchain_community.llms import Ollama


In [7]:
llm = Ollama(model="llama3.1:latest")

In [8]:
prompt = '''
You are an translation assistant for language translation tasks. You need to translate english sentence to vietnamese and response only the Vietnamese result.

English :{english}
Vietnamese: 

'''

In [9]:
def question_llm(english):
  prompt_formatted = prompt.format(english=english)
  res = llm.invoke(prompt_formatted)
  # print(prompt_formatted)
  return res

In [10]:
question_llm("a photography of a flooded street with a truck driving through it")

'Một bức ảnh chụp một con đường bị ngập nước với một chiếc xe tải đang chạy qua nó.'

In [11]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

bge_m3_ef = BGEM3EmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu', # Specify the device to use, e.g., 'cpu' or 'cuda:0'
    use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 215830.39it/s]
/Volumes/home/Learn/Master/3. Multimedia/AIChallenge/Code/.venv/lib/python3.12/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open

In [12]:
docs= [f"Hình ảnh đường phố bị ngập nước với một xe tải đang chạy qua."]
docs_embeddings = bge_m3_ef.encode_documents(docs)

vectors = docs_embeddings["dense"]

In [13]:
vectors

[array([-0.04253798,  0.00780115, -0.03375866, ...,  0.0280426 ,
        -0.04685987, -0.02310734], dtype=float32)]

In [14]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530"
)

In [15]:
if client.has_collection(collection_name="ai_challenge_collection"):
    client.drop_collection(collection_name="ai_challenge_collection")
client.create_collection(
        collection_name="ai_challenge_collection",
        dimension=1024,  # The vectors we will use in this demo has 768 dimensions
        auto_id=True
)


In [16]:
data = []
data.append({ "vector": vectors[0], "text": docs[0]} )
res = client.insert(collection_name="ai_challenge_collection", data=data)        
print(res)


{'insert_count': 1, 'ids': [452540126240705759]}


In [17]:

res = client.query(
    collection_name="ai_challenge_collection",  # target collection
    filter='',  # number of returned entities
    limit=100,
    output_fields=["id", 'text'],  # specifies fields to be returned
    # sorted=True, #
)
for item in res:
    print(item)

In [18]:
queries = ["Tìm ảnh thành phố ngập nước có một xe tải đang chạy"]

query_embeddings = bge_m3_ef.encode_queries(queries)

res = client.search(
    collection_name="ai_challenge_collection",  # target collection
    data=[query_embeddings['dense'][0]],  # query vectors
    limit=5,  # number of returned entities
    output_fields=["id","text"],  # specifies fields to be returned
)
context_items = res[0]

print(context_items)



[{'id': 452540126240705759, 'distance': 0.8477668762207031, 'entity': {'id': 452540126240705759, 'text': 'Hình ảnh đường phố bị ngập nước với một xe tải đang chạy qua.'}}]


In [19]:
def detect_caption(img_url):
  
  # img_url = 'keyframes/L01_V001/016.jpg' 
  # raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
  raw_image = Image.open(img_url).convert('RGB')

  # conditional image captioning
  text = "a photography of"
  inputs = processor(raw_image, text, return_tensors="pt",max_length=50)
  out = model.generate(**inputs)
  caption_english = processor.decode(out[0], skip_special_tokens=True)
  # caption_vietnamese = question_llm(caption_english)
  return caption_english


In [38]:
import os
import csv
import pandas as pd


def save_to_file(folder, keyframe_info):
  # Define the output CSV file path using the folder name
  csv_file_path = os.path.join('captions', f"{folder}.csv")
  df = pd.DataFrame(keyframe_info)
  df.to_csv(csv_file_path, index=False)  # index=False prevents the index from being written to the file


  print(f"Data from folder '{folder}' has been saved to {csv_file_path}")

In [44]:

# Define the root directory
root_dir = 'keyframes'
captions_dir = 'captions'

# Initialize a list to store the collected information
os.listdir(root_dir)

# Loop through all folders and images
for folder in sorted(os.listdir(root_dir)):
    csv_file_path = os.path.join(captions_dir, f"{folder}.csv")
    if os.path.exists(csv_file_path):
        print(f"CSV file for folder '{folder}' already exists. Skipping...")
        continue  # Skip to the next folder if CSV file already exists
    keyframe_info = []
    folder_path = os.path.join(root_dir, folder)
    if os.path.isdir(folder_path):
        for image in sorted(os.listdir(folder_path)):
            if image.endswith('.jpg'):
                img_url = f"keyframes/{folder}/{image}"
                caption = detect_caption(img_url)
                new_item = {
                    "folder": folder,
                    "frame": image,
                    "caption": caption
                }
                keyframe_info.append(new_item)
                print(new_item)
                
        save_to_file(folder,keyframe_info)

        print(keyframe_info)


CSV file for folder 'L01_V000' already exists. Skipping...
CSV file for folder 'L01_V0000' already exists. Skipping...


KeyboardInterrupt: 